# XAI Metrics based on Surrogate Model

In [ ]:
from holisticai.utils import BinaryClassificationProxy
from sklearn.ensemble import RandomForestClassifier

from holisticai.datasets import load_dataset

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm


from holisticai.inspection import compute_permutation_importance, compute_conditional_permutation_feature_importance
from holisticai.explainability.metrics import classification_surrogate_explainability_metrics
from holisticai.utils.data_preprocessor import simple_preprocessor
from holisticai.efficacy.metrics import classification_efficacy_metrics

# Regression (US Crime Dataset)

In [7]:
ds = load_dataset('us_crime', preprocessed=False)
dataset = ds.train_test_split(test_size=0.2, random_state=42)

train = dataset['train']
test = dataset['test']

Xt_train, Xt_test, yt_train, yt_test = simple_preprocessor(train['X'], test['X'], train['y'], test['y'], categorical_types=['object'], model_type='regression', only_numerics=True)

Train and create proxy model

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from holisticai.utils import RegressionProxy
from holisticai.efficacy.metrics import regression_efficacy_metrics
from holisticai.utils import concatenate_metrics

model = GradientBoostingRegressor()
model.fit(Xt_train, yt_train)
proxy = RegressionProxy(predict=model.predict)

y_pred_test = model.predict(Xt_test)
efficacy_metrics =regression_efficacy_metrics(yt_test, y_pred_test)
efficacy_metrics

,Value,Reference
Metric,,
RMSE,1.389627e-01,0
MAE,8.966271e-02,0
MAPE,1.374844e+12,0
Max Error,7.582146e-01,0
SMAPE,2.354820e-01,0


Create surrogate model

In [10]:
from holisticai.utils.surrogate_models import create_surrogate_model

y_train_pred = proxy.predict(Xt_train)    
surrogate = create_surrogate_model(Xt_train, y_train_pred, surrogate_type="shallow_tree", learning_task='regression')
surrogate

{"dtype":"Surrogate Model","attributes":{"Learning Task":"regression","Name":"Shallow Decision Tree Regressor","Model Type":"shallow_tree"}}

compute metrics based on surrogate

In [12]:
from holisticai.explainability.metrics import regression_surrogate_explainability_metrics

y_test_pred = proxy.predict(Xt_test)
surrogate_metrics = regression_surrogate_explainability_metrics(Xt_test, yt_test, y_test_pred, surrogate)
surrogate_metrics

,Value,Reference
MSE Degradation,0.220583,0
Surrogate Fidelity,0.7727,1
Surrogate Feature Stability,0.303333,1
